In [ ]:
import matplotlib.pyplot as plt
from nwm import NwmHs

# get data from National water model HydroShare App
dataset = NwmHs().get_data()

# show metadata
dataset.attrs


In [ ]:
# plot data
dataset.plot()
plt.ylabel('{}({})'.format(dataset.variable_unit_name,dataset.variable_unit))
plt.title('{}; {}'.format(dataset.archive.upper(), dataset.site_name))